In [2]:
# !pip install openai colorama

## Importing dependencies and Defining ChatGroq model

In [2]:
import os
import re
import math
import json

from openai import OpenAI
from dotenv import load_dotenv
from langchain_groq import ChatGroq  
import requests

load_dotenv()
groq_api_key=os.getenv("GROQ_API_KEY")
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")
os.environ['TAVILY_API_KEY'] = TAVILY_API_KEY



class ChatGroq:
    def __init__(self, groq_api_key: str, model: str = "llama-3.1-8b-instant", temperature: float = 0.0):
        self.api_key = groq_api_key
        self.model = model
        self.temperature = temperature
        self.base_url = "https://api.groq.com/openai/v1/chat/completions"

    def chat(self, messages: list[dict]) -> str:
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json",
        }

        payload = {
            "model": self.model,
            "messages": messages,
            "temperature": self.temperature,
        }

        response = requests.post(self.base_url, headers=headers, json=payload)

        # Raise exception if unauthorized
        if response.status_code == 401:
            print("❌ Unauthorized — check your GROQ_API_KEY")
            print("Full response:", response.text)
            raise Exception("GROQ API key invalid or not authorized.")

        response.raise_for_status()
        return response.json()["choices"][0]["message"]["content"]
model = ChatGroq(groq_api_key=groq_api_key)


## Defining utility functions

In [3]:
# @title
"""
This is a collection of helper functions and methods we are going to use in
the Agent implementation. You don't need to know the specific implementation
of these to follow the Agent code. But, if you are curious, feel free to check
them out.
"""

import re
import time

from colorama import Fore
from colorama import Style

from dataclasses import dataclass


def completions_create(client, messages: list, model: str) -> str:
    """
    Sends a request to the Groq client to interact with the llama-3.1-8b-instant model.

    Args:
        client (ChatGroq): The custom ChatGroq client object
        messages (list[dict]): A list of message objects containing chat history for the model.
        model (str): The model name (not used here, kept for compatibility)

    Returns:
        str: The content of the model's response.
    """
    return client.chat(messages)



def build_prompt_structure(prompt: str, role: str, tag: str = "") -> dict:
    """
    Builds a structured prompt that includes the role and content.

    Args:
        prompt (str): The actual content of the prompt.
        role (str): The role of the speaker (e.g., user, assistant).

    Returns:
        dict: A dictionary representing the structured prompt.
    """
    if tag:
        prompt = f"<{tag}>{prompt}</{tag}>"
    return {"role": role, "content": prompt}

def update_chat_history(history: list, msg: str, role: str):
    """
    Updates the chat history by appending the latest response.

    Args:
        history (list): The list representing the current chat history.
        msg (str): The message to append.
        role (str): The role type (e.g. 'user', 'assistant', 'system')
    """
    history.append(build_prompt_structure(prompt=msg, role=role))


class ChatHistory(list):
    def __init__(self, messages: list | None = None, total_length: int = -1):
        """Initialise the queue with a fixed total length.

        Args:
            messages (list | None): A list of initial messages
            total_length (int): The maximum number of messages the chat history can hold.
        """
        if messages is None:
            messages = []

        super().__init__(messages)
        self.total_length = total_length

    def append(self, msg: str):
        """Add a message to the queue.

        Args:
            msg (str): The message to be added to the queue
        """
        if len(self) == self.total_length:
            self.pop(0)
        super().append(msg)



class FixedFirstChatHistory(ChatHistory):
    def __init__(self, messages: list | None = None, total_length: int = -1):
        """Initialise the queue with a fixed total length.

        Args:
            messages (list | None): A list of initial messages
            total_length (int): The maximum number of messages the chat history can hold.
        """
        super().__init__(messages, total_length)

    def append(self, msg: str):
        """Add a message to the queue. The first messaage will always stay fixed.

        Args:
            msg (str): The message to be added to the queue
        """
        if len(self) == self.total_length:
            self.pop(1)
        super().append(msg)

def fancy_print(message: str) -> None:
    """
    Displays a fancy print message.

    Args:
        message (str): The message to display.
    """
    print(Style.BRIGHT + Fore.CYAN + f"\n{'=' * 50}")
    print(Fore.MAGENTA + f"{message}")
    print(Style.BRIGHT + Fore.CYAN + f"{'=' * 50}\n")
    time.sleep(0.5)


def fancy_step_tracker(step: int, total_steps: int) -> None:
    """
    Displays a fancy step tracker for each iteration of the generation-reflection loop.

    Args:
        step (int): The current step in the loop.
        total_steps (int): The total number of steps in the loop.
    """
    fancy_print(f"STEP {step + 1}/{total_steps}")


@dataclass
class TagContentResult:
    """
    A data class to represent the result of extracting tag content.

    Attributes:
        content (List[str]): A list of strings containing the content found between the specified tags.
        found (bool): A flag indicating whether any content was found for the given tag.
    """

    content: list[str]
    found: bool


def extract_tag_content(text: str, tag: str) -> TagContentResult:
    """
    Extracts all content enclosed by specified tags (e.g., <thought>, <response>, etc.).

    Parameters:
        text (str): The input string containing multiple potential tags.
        tag (str): The name of the tag to search for (e.g., 'thought', 'response').

    Returns:
        dict: A dictionary with the following keys:
            - 'content' (list): A list of strings containing the content found between the specified tags.
            - 'found' (bool): A flag indicating whether any content was found for the given tag.
    """
    # Build the regex pattern dynamically to find multiple occurrences of the tag
    tag_pattern = rf"<{tag}>(.*?)</{tag}>"

    # Use findall to capture all content between the specified tag
    matched_contents = re.findall(tag_pattern, text, re.DOTALL)

    # Return the dataclass instance with the result
    return TagContentResult(
        content=[content.strip() for content in matched_contents],
        found=bool(matched_contents),
    )


## Approach 
#### Planning: Using LLM for Research Question Generation 
-  The agent will use an LLM (for example: OpenAI's GPT or Gemini or Ollama for local LLM to assist in reasoning. The agent first asks the LLM to generate a list of research questions related to the given topic. 
- The LLM should suggest 5 to 6 well-structured questions that cover different aspects of the topic. 
- Example: If the topic is "Climate Change," the LLM may generate questions like: 
  1. What are the main causes of climate change? 
  2. How has global temperature changed over the past century? 
  3. What policies are in place to combat climate change? 

#### Acting: Web Search for Gathering Information 
- The agent will take each research question and search the web to find relevant and recent information. 
- The agent should extract key points from search results and store them. 

#### Report Generation 
- After collecting data for all questions, the agent will compile a structured report summarizing its findings. 
- The report should have a title, an introduction, sections for each research question, and a conclusion

In [ ]:
import json
from colorama import Fore
from typing import Union


class ReactAgent:
    def __init__(
        self,
        client: object,
        model: str = "llama-3.1-8b-instant",
    ):
        self.client = client
        self.model = model
        self.questions = []
        self.answers = {}

    def plan_questions(self, topic: str) -> list[str]:
        fancy_print("🔍 Planning Phase: Generating Research Questions")
        prompt = (
            f"You are a helpful research assistant.\n"
            f"Generate 5–6 research questions on the topic: '{topic}'.\n"
            f"Each question should explore a different aspect of the topic.\n"
            f"Present them as a numbered list.\n"
            f"Only have the questions and not the descrptions of the questions"
        )

        messages = [
            build_prompt_structure("You are a helpful assistant.", "system"),
            build_prompt_structure(prompt, "user"),
        ]

        response = completions_create(self.client, messages, self.model)

        # Extract and clean questions
        self.questions = [
            q.strip().lstrip("0123456789. ").strip()
            for q in response.split("\n")
            if q.strip()
        ]
        print(Fore.YELLOW + "\nGenerated Questions:")
        for q in self.questions:
            print("•", q)

        return self.questions

    def search_answers(self, web_search_fn) -> dict:
        fancy_print("🌐 Acting Phase: Searching the Web for Each Question")

        for idx, question in enumerate(self.questions):
            print(Fore.CYAN + f"\n🔎 Q{idx + 1}: {question}")
            results = web_search_fn(question)
            self.answers[question] = results
            for r in results:
                print(Fore.GREEN + "•", r)

        return self.answers

    def compile_report(self, topic: str) -> str:
        fancy_print("📝 Compiling Final Structured Report")

        # Start building report
        report = f"# Research Report on '{topic}'\n\n"
        report += "## Introduction\n"
        report += f"This report presents findings on key questions related to **{topic}**, using real-time web research.\n\n"

        body_summary = ""
        for question in self.questions:
            report += f"## {question}\n"
            q_answers = self.answers.get(question, [])
            for point in q_answers:
                report += f"- {point}\n"
            report += "\n"
            body_summary += f"{question}\n" + "\n".join(f"- {ans}" for ans in q_answers) + "\n\n"

        # Ask LLM to write a conclusion based on the compiled Q&A
        fancy_print("🤖 Generating LLM-based Conclusion")

        messages = [
            build_prompt_structure("You are a research assistant. Summarize findings in bullet points.", "system"),
            build_prompt_structure(
                f"Based on the following research questions and answers, generate a conclusion in 4–5 key bullet points:\n\n{body_summary}",
                "user"
            )
        ]

        conclusion_response = completions_create(self.client, messages, self.model)

        # Add conclusion to report
        report += "## Conclusion\n"
        report += conclusion_response.strip() + "\n"

        return report


    def run(self, topic: str, web_search_fn) -> str:
        self.plan_questions(topic)
        self.search_answers(web_search_fn)
        return self.compile_report(topic)


#### Defining the web search using Tavily

In [11]:
# web_search.py
import requests
import os


def search_web(query: str) -> list[str]:
    """
    Perform a web search using the Tavily API and return summaries of results.
    """
    if not TAVILY_API_KEY:
        raise ValueError("TAVILY_API_KEY is not set in your environment.")

    url = "https://api.tavily.com/search"
    headers = {
        "Authorization": f"Bearer {TAVILY_API_KEY}",
        "Content-Type": "application/json"
    }

    payload = {
        "query": query,
        "search_depth": "basic",
        "max_results": 3,
        "include_answer": False,
        "include_raw_content": False
    }

    response = requests.post(url, headers=headers, json=payload)
    response.raise_for_status()

    data = response.json()
    summaries = []
    for result in data.get("results", []):
        title = result.get("title", "No Title")
        snippet = result.get("content", "No Summary")
        summaries.append(f"{title}: {snippet}")

    return summaries


In [12]:

if __name__ == "__main__":
    topic='Climate Change'
    client = ChatGroq(groq_api_key=groq_api_key, model="llama-3.1-8b-instant")
    agent = ReactAgent(client=client)

    report = agent.run(topic, web_search_fn=search_web)

    print("\n" + "="*60)
    print(report)
    print("="*60)



🔍 Planning Phase: Generating Research Questions




Generated Questions:
• What are the primary drivers of climate change, and how do they interact with one another?
• How does climate change impact global food security, and what strategies can be implemented to mitigate these effects?
• What are the economic costs and benefits of transitioning to renewable energy sources to combat climate change?
• How do climate change and urbanization intersect, and what are the implications for urban planning and infrastructure development?
• What are the effects of climate change on global health, particularly in vulnerable populations such as the elderly and young children?
• Can geoengineering technologies be a viable solution to mitigate the effects of climate change, and what are the potential risks and benefits?

🌐 Acting Phase: Searching the Web for Each Question


🔎 Q1: What are the primary drivers of climate change, and how do they interact with one another?
• Causes and Effects of Climate Change | United Nations: Causes and Effects of Cli

## FINAL REPORT

In [13]:
from IPython.display import Markdown
# Display the text
Markdown(report)


# Research Report on 'Climate Change'

## Introduction
This report presents findings on key questions related to **Climate Change**, using real-time web research.

## What are the primary drivers of climate change, and how do they interact with one another?
- Causes and Effects of Climate Change | United Nations: Causes and Effects of Climate Change | United Nations Fossil fuels – coal, oil and gas – are by far the largest contributor to global climate change, accounting for over 75 per cent of global greenhouse gas emissions and nearly 90 per cent of all carbon dioxide emissions. Producing food causes emissions of carbon dioxide, methane, and other greenhouse gases in various ways, including through deforestation and clearing of land for agriculture and grazing, digestion by cows and sheep, the production and use of fertilizers and manure for growing crops, and the use of energy to run farm equipment or fishing boats, usually with fossil fuels. Climate change poses risks to the survival of species on land and in the ocean.
- Causes of Climate Change | US EPA: _Human and natural factors both influence the earth’s climate, but the long-term trend observed over the past century can only be explained by the effect of human activities on climate.Source: U.S. Global Change Research Program, Fourth National Climate Assessment, Chapter 2: Our Changing Climate,2018._ Since the Industrial Revolution, human activities have released large amounts of carbon dioxide and other greenhouse gases into the atmosphere, which has changed the earth’s climate. During cool glacial periods, carbon dioxide levels were lower.25The heating or cooling of the earth’s surface and oceans can cause changes in the natural sources and sinks of these gases, and thus change greenhouse gas concentrations in the atmosphere.26These changing concentrations have acted as a positive climate feedback, amplifying the temperature changes caused by long-term shifts in the earth’s orbit.27
- What Are the Causes of Climate Change? - NRDC: (This type of climate change is sometimes referred to as _anthropogenic_, which is just a way of saying “caused by human beings.”) The unchecked burning of fossil fuels over the past 150 years has drastically increased the presence of atmospheric greenhouse gases, most notably carbon dioxide. Because of the electricity sector’s historical investment in these dirty energy sources, it accounts for roughly a quarter of U.S. greenhouse gas emissions, including carbon dioxide, methane, and nitrous oxide. The first step in reducing it is for us to acknowledge the uneven distribution of climate change’s causes and effects, and for those who bear the greatest responsibility for global greenhouse gas emissions to slash them without bringing further harm to those who are least responsible.

## How does climate change impact global food security, and what strategies can be implemented to mitigate these effects?
- How to mitigate the effects of climate change on global food security: The effects of climate change on food security are particularly apparent in developing countries and can be seen in both the health of their citizens as well as the livelihoods of small farmers. When food availability is degraded due to changing climate conditions such as drought or flooding, there is a risk of malnutrition among vulnerable populations who struggle to access nutritious foods.
- Climate Explainer: Food Security and Climate Change: **What is the state of global food security today, and what is the role of climate change?** **How might climate change affect farming and food security in the future?** For areas of the world that are already water-constrained, climate change will increasingly cause adverse impacts on agricultural production through diminishing water supplies, increases in extreme events like floods and severe storms, heat stress, and increased prevalence of pests and diseases. **What is the World Bank doing to help countries build food security in the face of climate change?** The World Bank Group’s Climate Change Action Plan (2021-2025) is stepping up support for climate-smart agriculture across the agriculture and food value chains and via policy and technological interventions to enhance productivity, improve resilience, and reduce GHG emissions. Website: World Bank - Climate Change Website: World Bank - Climate Change
- Climate Change, Global Food Security, and the U.S. Food System: Climate Change, Global Food Security, and the U.S. Food System | Home *   Food Safety Overview *   Food Assistance Programs Overview and emergency food assistance among many other programs.") *   Food Safety Overview *   Food Assistance Programs Overview and emergency food assistance among many other programs.") The U.S. Department of Agriculture is looking to military veterans across the country to fill the roles that keep America’s food supply safe and secure, preserve and strengthen rural communities, and restore and conserve the environment. *   Marketing and Trade Assistance Overview *   Marketing and Trade Assistance Overview *   Working with USDA Overview *   Reports and Data Overview *   USDA Policies Overview *   Working with USDA Overview *   Reports and Data Overview *   USDA Policies Overview

## What are the economic costs and benefits of transitioning to renewable energy sources to combat climate change?
- The Energy Transition in 2025: What to Watch For - RMI: *   Description The yt-remote-fast-check-period cookie is used by YouTube to store the user's video player preferences for embedded YouTube videos. *   Description Twitter sets this cookie to integrate and share features for social media and also store information about how the user uses the website, for tracking and targeting. *   Description YouTube sets this cookie to measure bandwidth, determining whether the user gets the new or old player interface. *   **New year, new progress?** Energy efficiency and methane are the two fastest ways to cut warming, but also the two furthest off track. And from vehicles to heat pumps to industry, annual electrification progress doubled in the past year — a key step for the energy efficiency pledge and its many benefits.
- Renewable energy – powering a safer future | United Nations: Renewable energy – powering a safer future | United Nations *   Renewable Energy 1. Renewable energy sources are all around us Switching to clean sources of energy, such as wind and solar, thus helps address not only climate change but also air pollution and health. In comparison, about $4.5 trillion a year needs to be invested in renewable energy until 2030 – including investments in technology and infrastructure – to allow us to reach net-zero emissions by 2050. ### What is renewable energy? Learn more about the differences between fossil fuels and renewables, the benefits of renewable energy, and how we can act now. UN Secretary-General outlines five critical actions the world needs to prioritize now to speed up the global shift to renewable energy. *   What is renewable energy
- The economic opportunities of the clean energy transition: The research finds that government policies that accelerate the cleaner energy transition to net zero will reduce future energy bills and create jobs.

## How do climate change and urbanization intersect, and what are the implications for urban planning and infrastructure development?
- how does urban planning contribute to climate change? | Clarity: Urban planning decisions influence how cities grow, impacting emissions and resilience to climate impacts. By implementing environmental master plans, cities can reduce emissions and enhance resilience, making urban planning a critical tool in the global climate strategy. Exploring the trends in the Smart City industry, such as citizen empowerment and sustainability, can help urban planners understand how to improve resource efficiency and sustainability in urban areas. Leveraging innovative approaches to improve air quality in urban settings ensures that urban areas can adapt to climate challenges while promoting sustainability. Additionally, understanding the impact of air quality management in urban planning can further enhance city resilience. Urban planning drives climate change solutions through city design.
- Climate Change | UN-Habitat: The effects of urbanization and climate change are converging in dangerous ways. Global warming is likely to reach 1.5°C between 2030 and 2052,
- The Intersection of Sustainability and Resilience in Infrastructure: With infrastructure both sustainability and resilience seek to reduce environmental impact and conserve resources in the short and long term.

## What are the effects of climate change on global health, particularly in vulnerable populations such as the elderly and young children?
- Global Climate Change and Children's Health: Threats and ...: Global climate change will have multiple effects on human health. Vulnerable populations—children, the elderly, and the poor—will be disproportionately affected
- Climate Change and Children's Health | US EPA: Increases in average and extreme temperatures are expected to lead to more heat illnesses and deaths among vulnerable groups, including children
- Experts warn of serious health impacts from climate change for ...: Pregnant women, newborns, children, adolescents and older people are facing serious health complications due to climate change, according to a new collection

## Can geoengineering technologies be a viable solution to mitigate the effects of climate change, and what are the potential risks and benefits?
- Why Geoengineering is a False Solution to the Climate Crisis: Geoengineering is fraught with risks, uncertainties, and dangers that threaten to delay real climate action and cause more harm than good.
- Geoengineering could fight climate change—if scientists ... - Science: Geoengineering could be crucial in the fight against climate change. But first scientists need to learn how to talk to the public about it.
- New study compares the benefits and risks of solar geoengineering: It finds the benefits—in the form of reduced heat-induced mortality—from using solar geoengineering are about ten times larger than the

## Conclusion
Based on the research findings, here are 4-5 key bullet points summarizing the conclusions:

• **Climate Change is Primarily Driven by Human Activities**: The research confirms that human activities, particularly the burning of fossil fuels, deforestation, and agriculture, are the primary drivers of climate change, accounting for over 75% of global greenhouse gas emissions.

• **Climate Change Impacts Global Food Security and Health**: Climate change poses significant risks to global food security, particularly in developing countries, and has severe health implications for vulnerable populations, including children, the elderly, and the poor.

• **Transitioning to Renewable Energy is Crucial**: The research highlights the need to transition to renewable energy sources to combat climate change, with benefits including reduced energy bills, job creation, and improved air quality. Investing in renewable energy is essential to reach net-zero emissions by 2050.

• **Urbanization and Climate Change Intersect**: The effects of urbanization and climate change are converging, with urban planning playing a critical role in reducing emissions and enhancing resilience. Cities must adopt sustainable and resilient infrastructure development to adapt to climate challenges.

• **Geoengineering Technologies are Not a Viable Solution**: While geoengineering technologies may offer some benefits, they are fraught with risks, uncertainties, and dangers that threaten to delay real climate action and cause more harm than good. A more effective approach is to focus on reducing greenhouse gas emissions through renewable energy, energy efficiency, and sustainable land use practices.
